In [36]:
import importlib
import wrf_management.project_global_constants as gc
import wrf_management.ungrib as un
import os
import pandas as pd
import subprocess as su
import wrf_management.run_utilities as ru
import wrf_management.real as re

In [46]:
print(gc.RUN_NAME)
job = 'real'
met_pref='metgrid'
real = False

LIST_S_LINKS = [
    'real.exe',
    #     'link_grib.csh',
#     'metgrid'
]

LIST_H_LINKS = [
    #     'Vtable',
    'env_WRFv4.bash'
]

run_2019_02_20


In [26]:
# con = sq.connect(gc.PATH_DB)
gc.PATH_DB

'/tmp/wrf_management/db_folder/wrf_man.sqlite'

In [27]:
importlib.reload(un)
importlib.reload(ru)
run_row = ru.get_run_row()
print(run_row)

index                             1
run_name             run_2019_02_20
geogrid_path                geogrid
config_path          run_2019_02_20
data_path       runs/run_2019_02_20
Name: 0, dtype: object


In [28]:
run_path = os.path.join(gc.PATH_DATA, run_row.data_path)
print(run_path)

/tmp/wrf_management/data_folder/runs/run_2019_02_20


In [29]:
job_row = ru.get_next_row(job=job)
print(job_row)

date              2017-12-02 00:00:00
ungrib_surf                       100
ungrib_press                      100
ungrib_lake                         0
ungrib_avgtsfc                    100
metgrid                           100
real                                0
Name: 0, dtype: object


In [30]:
if real:
    ru.update_run_table(val=job_row[job] + 1,
                        job=job,
                        date=job_row['date']
                        )

job_path = ru.getmk_job_path(run_row, job_row, job)
print(job_path)

/tmp/wrf_management/data_folder/runs/run_2019_02_20/2017_12_02/real


In [31]:
conf_path = ru.get_conf_path(run_row)
print(conf_path)

/Users/diego/wrf_management/wrf_management/config_dir/run_2019_02_20


'2017-12-02 00:00:00'

In [44]:
duration_h = 24
importlib.reload(re)
re.skim_namelist_copy_real(
    conf_path, job_path, date = job_row.date, 
    duration_h=duration_h
);

In [50]:
importlib.reload(re)


re.link_met_files(
    job_path=job_path,
    met_pref=met_pref
)

In [51]:
# if gc.ID=='taito_login':
ru.copy_hard_links(conf_path, job_path, LIST_H_LINKS)
ru.copy_soft_links(
    os.path.join(gc.PATH_WPS, ''),
    job_path, LIST_S_LINKS)

env_WRFv4.bash
real.exe


In [68]:
run_script = \
    """#!/bin/bash
    cd {job_path}
    source ./env_WRFv4.bash 
    ./metgrid.exe
    exit $?
    """.format(job_path=job_path)
print(run_script)
bs_path = os.path.join(job_path, 'run_me.sh')
bs_file = open(bs_path, 'w')
bs_file.write(run_script)
bs_file.close()

#!/bin/bash
    cd /wrk/aliagadi/DONOTREMOVE/wrf_management_data/runs/run_2019_02_20/2017_12_03/ungrib_avgtsfc
    source ./env_WRFv4.bash 
    ./avg_tsfc.exe > avg_tsfc.log
    exit $?
    


In [69]:
if gc.ID == 'taito_login':
    res = su.run(['/bin/bash', bs_path], stdout=su.PIPE, stderr=su.PIPE)

if gc.ID == 'taito_login' and res.returncode == 0:
    un.update_run_table(val=100,
                        job=job,
                        date=job_row['date']
                        )